In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
import warnings
warnings.filterwarnings('ignore')
import scorecardpy as sc
import pprint 

pd.set_option('display.max_columns', None)

In [2]:
full_data = pd.read_csv('IS453 Group Assignment - Data.csv')

#use a copy of hmeq_data for credit risk model
full_data = full_data.copy()

full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   STATUS                       307511 non-null  int64  
 1   NAME_CONTRACT_TYPE           307511 non-null  object 
 2   CODE_GENDER                  307511 non-null  object 
 3   FLAG_OWN_CAR                 307511 non-null  object 
 4   FLAG_OWN_REALTY              307511 non-null  object 
 5   CNT_CHILDREN                 307511 non-null  int64  
 6   AMT_INCOME_TOTAL             307511 non-null  float64
 7   AMT_CREDIT                   307511 non-null  float64
 8   AMT_ANNUITY                  307499 non-null  float64
 9   AMT_GOODS_PRICE              307233 non-null  float64
 10  NAME_TYPE_SUITE              306219 non-null  object 
 11  NAME_INCOME_TYPE             307511 non-null  object 
 12  NAME_EDUCATION_TYPE          307511 non-null  object 
 13 

In [3]:
working_set = full_data.copy()
working_set_orig = full_data.copy()

working_set.head()

,STATUS,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,NaN,0.650442,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,Religion,NaN,0.322738,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#Feature Extraction -- Combinding the Flag Documents columns 
working_set['TOTAL_FLAG_DOCUMENTS'] = working_set['FLAG_DOCUMENT_2'] + working_set['FLAG_DOCUMENT_3'] + working_set['FLAG_DOCUMENT_4'] + working_set['FLAG_DOCUMENT_5'] + working_set['FLAG_DOCUMENT_6'] + working_set['FLAG_DOCUMENT_7'] + working_set['FLAG_DOCUMENT_8'] + working_set['FLAG_DOCUMENT_9'] + working_set['FLAG_DOCUMENT_10'] + working_set['FLAG_DOCUMENT_11'] + working_set['FLAG_DOCUMENT_12'] + working_set['FLAG_DOCUMENT_13'] + working_set['FLAG_DOCUMENT_14'] + working_set['FLAG_DOCUMENT_15'] + working_set['FLAG_DOCUMENT_16'] + working_set['FLAG_DOCUMENT_17'] + working_set['FLAG_DOCUMENT_18'] + working_set['FLAG_DOCUMENT_19'] + working_set['FLAG_DOCUMENT_20'] + working_set['FLAG_DOCUMENT_21']

working_set['TOTAL_FLAG_DOCUMENTS'].head()

working_set.drop('FLAG_DOCUMENT_2', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_3', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_4', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_5', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_6', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_7', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_8', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_9', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_10', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_11', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_12', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_13', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_14', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_15', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_16', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_17', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_18', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_19', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_20', inplace=True, axis=1)
working_set.drop('FLAG_DOCUMENT_21', inplace=True, axis=1)

working_set.head(3)

,STATUS,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TOTAL_FLAG_DOCUMENTS
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,2.0,2.0,2.0,2.0,-1134.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,1.0,0.0,1.0,0.0,-828.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,0.0,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# 1. Data Preparation 


In [5]:
#Dropping Variables that result in biasedness and highly correlated ones
drop_col = ['OBS_60_CNT_SOCIAL_CIRCLE', 'AMT_GOODS_PRICE','REGION_RATING_CLIENT_W_CITY','CNT_FAM_MEMBERS',
            'LIVE_REGION_NOT_WORK_REGION','DEF_60_CNT_SOCIAL_CIRCLE','LIVE_CITY_NOT_WORK_CITY', 'AMT_GOODS_PRICE',
            'AMT_ANNUITY', 'CODE_GENDER']
working_set.drop(columns= drop_col, inplace= True)

working_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 47 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   STATUS                      307511 non-null  int64  
 1   NAME_CONTRACT_TYPE          307511 non-null  object 
 2   FLAG_OWN_CAR                307511 non-null  object 
 3   FLAG_OWN_REALTY             307511 non-null  object 
 4   CNT_CHILDREN                307511 non-null  int64  
 5   AMT_INCOME_TOTAL            307511 non-null  float64
 6   AMT_CREDIT                  307511 non-null  float64
 7   NAME_TYPE_SUITE             306219 non-null  object 
 8   NAME_INCOME_TYPE            307511 non-null  object 
 9   NAME_EDUCATION_TYPE         307511 non-null  object 
 10  NAME_FAMILY_STATUS          307511 non-null  object 
 11  NAME_HOUSING_TYPE           307511 non-null  object 
 12  REGION_POPULATION_RELATIVE  307511 non-null  float64
 13  DAYS_BIRTH    

In [6]:
# Inspect rows missing more than 4 values &  make sure not to reduce sample size too much 
rows_w_gt_3_na = working_set[working_set.isnull().sum(axis=1) > 3].shape[0]
print('Percent of total rows missing more than 3 values: ' + str("{:.1%}".format(rows_w_gt_3_na/working_set.shape[0])))
print(rows_w_gt_3_na)

Percent of total rows missing more than 3 values: 22.4%
68762


In [7]:
working_set.dropna(thresh=working_set.shape[1]-3,inplace=True)
print("Starting row count: " + str(working_set_orig.shape[0]))
print("Ending row count: " + str(working_set.shape[0]))
print("Percent dropped: " + "{:.2%}".format(1-working_set.shape[0]/working_set_orig.shape[0]))

Starting row count: 307511
Ending row count: 238749
Percent dropped: 22.36%


In [8]:
#Check % of missing column values again with revised dataset 
col_w_na = pd.DataFrame((working_set.isnull().sum().sort_values(ascending=False)/working_set.shape[0])).applymap("{0:.0%}".format)

col_w_na.head(10)

,0
OWN_CAR_AGE,62%
EXT_SOURCE_1,51%
APARTMENTS_AVG,45%
OCCUPATION_TYPE,25%
EXT_SOURCE_3,5%
NAME_TYPE_SUITE,0%
DEF_30_CNT_SOCIAL_CIRCLE,0%
OBS_30_CNT_SOCIAL_CIRCLE,0%
EXT_SOURCE_2,0%
AMT_REQ_CREDIT_BUREAU_YEAR,0%


In [9]:
#Remove 3 variables as the amount of missing data is more than 50% 
working_set = working_set.drop(columns=['OWN_CAR_AGE','EXT_SOURCE_1'], axis=1)

working_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238749 entries, 0 to 307510
Data columns (total 45 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   STATUS                      238749 non-null  int64  
 1   NAME_CONTRACT_TYPE          238749 non-null  object 
 2   FLAG_OWN_CAR                238749 non-null  object 
 3   FLAG_OWN_REALTY             238749 non-null  object 
 4   CNT_CHILDREN                238749 non-null  int64  
 5   AMT_INCOME_TOTAL            238749 non-null  float64
 6   AMT_CREDIT                  238749 non-null  float64
 7   NAME_TYPE_SUITE             237985 non-null  object 
 8   NAME_INCOME_TYPE            238749 non-null  object 
 9   NAME_EDUCATION_TYPE         238749 non-null  object 
 10  NAME_FAMILY_STATUS          238749 non-null  object 
 11  NAME_HOUSING_TYPE           238749 non-null  object 
 12  REGION_POPULATION_RELATIVE  238749 non-null  float64
 13  DAYS_BIRTH    

In [10]:
#Remove missing values 
working_set.dropna(inplace=True)
working_set.isna().sum()

STATUS                        0
NAME_CONTRACT_TYPE            0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
CNT_CHILDREN                  0
AMT_INCOME_TOTAL              0
AMT_CREDIT                    0
NAME_TYPE_SUITE               0
NAME_INCOME_TYPE              0
NAME_EDUCATION_TYPE           0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
REGION_POPULATION_RELATIVE    0
DAYS_BIRTH                    0
DAYS_EMPLOYED                 0
DAYS_REGISTRATION             0
DAYS_ID_PUBLISH               0
FLAG_MOBIL                    0
FLAG_EMP_PHONE                0
FLAG_WORK_PHONE               0
FLAG_CONT_MOBILE              0
FLAG_PHONE                    0
FLAG_EMAIL                    0
OCCUPATION_TYPE               0
REGION_RATING_CLIENT          0
WEEKDAY_APPR_PROCESS_START    0
HOUR_APPR_PROCESS_START       0
REG_REGION_NOT_LIVE_REGION    0
REG_REGION_NOT_WORK_REGION    0
REG_CITY_NOT_LIVE_CITY        0
REG_CITY_NOT_WORK_CITY        0
ORGANIZA

In [11]:
working_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83445 entries, 0 to 307510
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   STATUS                      83445 non-null  int64  
 1   NAME_CONTRACT_TYPE          83445 non-null  object 
 2   FLAG_OWN_CAR                83445 non-null  object 
 3   FLAG_OWN_REALTY             83445 non-null  object 
 4   CNT_CHILDREN                83445 non-null  int64  
 5   AMT_INCOME_TOTAL            83445 non-null  float64
 6   AMT_CREDIT                  83445 non-null  float64
 7   NAME_TYPE_SUITE             83445 non-null  object 
 8   NAME_INCOME_TYPE            83445 non-null  object 
 9   NAME_EDUCATION_TYPE         83445 non-null  object 
 10  NAME_FAMILY_STATUS          83445 non-null  object 
 11  NAME_HOUSING_TYPE           83445 non-null  object 
 12  REGION_POPULATION_RELATIVE  83445 non-null  float64
 13  DAYS_BIRTH                  83

In [13]:
#Generate WOE Bins 

# automatically calculate bin ranges 
bins = sc.woebin(working_set, y='STATUS')

# make it easy to read the bins
for variables , bindetails in bins.items():
    print(variables , " : ")
    display(bindetails)
    print("--"*50)

[INFO] creating woe binning ...
>>> There are 1 variables have too many unique non-numberic values, which might cause the binning process slow. Please double check the following variables: 
ORGANIZATION_TYPE
>>> Continue the binning process?
1: yes 
2: no
Binning on 83445 rows and 44 columns in 00:00:23
NAME_EDUCATION_TYPE  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NAME_EDUCATION_TYPE,"Academic degree%,%Higher education",25830,0.309545,24509,1321,0.051142,-0.396824,0.041237,0.055959,"Academic degree%,%Higher education",False
1,NAME_EDUCATION_TYPE,"Incomplete higher%,%Secondary / secondary spec...",57615,0.690455,52744,4871,0.084544,0.141677,0.014723,0.055959,"Incomplete higher%,%Secondary / secondary spec...",False


----------------------------------------------------------------------------------------------------
NAME_TYPE_SUITE  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NAME_TYPE_SUITE,"Group of people%,%Other_B%,%Children%,%Family",11263,0.134975,10476,787,0.069875,-0.064786,0.000551,0.000634,"Group of people%,%Other_B%,%Children%,%Family",False
1,NAME_TYPE_SUITE,"Other_A%,%Spouse, partner%,%Unaccompanied",72182,0.865025,66777,5405,0.074880,0.009793,0.000083,0.000634,"Other_A%,%Spouse, partner%,%Unaccompanied",False


----------------------------------------------------------------------------------------------------
DEF_30_CNT_SOCIAL_CIRCLE  : 


...

----------------------------------------------------------------------------------------------------
CNT_CHILDREN  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CNT_CHILDREN,"[-inf,1.0)",53710,0.643657,49760,3950,0.073543,-0.009668,5.991886e-05,0.000201,1.0,False
1,CNT_CHILDREN,"[1.0,2.0)",20659,0.247576,19092,1567,0.075851,0.023721,1.407245e-04,0.000201,2.0,False
2,CNT_CHILDREN,"[2.0,inf)",9076,0.108766,8401,675,0.074372,0.002434,6.451863e-07,0.000201,inf,False


----------------------------------------------------------------------------------------------------
DAYS_BIRTH  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DAYS_BIRTH,"[-inf,-19500.0)",10912,0.130769,10399,513,0.047012,-0.485361,0.025123,0.056474,-19500.0,False
1,DAYS_BIRTH,"[-19500.0,-14000.0)",37466,0.448990,34969,2497,0.066647,-0.115544,0.005707,0.056474,-14000.0,False
2,DAYS_BIRTH,"[-14000.0,-11000.0)",21954,0.263095,20084,1870,0.085178,0.149843,0.006297,0.056474,-11000.0,False
3,DAYS_BIRTH,"[-11000.0,inf)",13113,0.157145,11801,1312,0.100053,0.327196,0.019347,0.056474,inf,False


----------------------------------------------------------------------------------------------------


In [35]:
pd.options.display.max_seq_items = 2000

for variables , bindetails in bins.items():
    if variables == 'OCCUPATION_TYPE':
        print(str(bindetails.bin[2]))

Cleaning staff%,%Sales staff%,%Laborers%,%Security staff%,%Cooking staff%,%Drivers%,%Waiters/barmen staff%,%Low-skill Laborers


In [17]:
scorecard_set = working_set[['STATUS', 'DAYS_EMPLOYED', 'OCCUPATION_TYPE', 'AMT_CREDIT', 'DAYS_BIRTH', 'ORGANIZATION_TYPE','NAME_EDUCATION_TYPE', 'EXT_SOURCE_2', 'EXT_SOURCE_3']] 

scorecard_set.head()


,STATUS,DAYS_EMPLOYED,OCCUPATION_TYPE,AMT_CREDIT,DAYS_BIRTH,ORGANIZATION_TYPE,NAME_EDUCATION_TYPE,EXT_SOURCE_2,EXT_SOURCE_3
0,1,-637,Laborers,406597.5,-9461,Business Entity Type 3,Secondary / secondary special,0.262949,0.139376
12,0,-2717,Laborers,80865.0,-13439,Business Entity Type 2,Secondary / secondary special,0.715042,0.176653
13,0,-3028,Drivers,918468.0,-14086,Self-employed,Secondary / secondary special,0.566907,0.770087
18,0,-7804,Laborers,157500.0,-17718,Housing,Secondary / secondary special,0.706428,0.556727
22,0,-4306,Sales staff,497520.0,-11146,Self-employed,Secondary / secondary special,0.457143,0.358951


In [26]:
#Train-test split 
train, test = sc.split_df(scorecard_set, 'STATUS', ratio=0.7).values()
print(train.shape)
print(test.shape)

(58411, 9)
(25034, 9)


In [27]:
# prepare a dataset with the WOE values for Logistic Regression training
# woebin_ply() converts original values of input data into woe
train_woe = sc.woebin_ply(train, bins)
test_woe = sc.woebin_ply(test, bins)
train_woe

[INFO] converting into woe values ...
[INFO] converting into woe values ...


,STATUS,NAME_EDUCATION_TYPE_woe,EXT_SOURCE_3_woe,DAYS_EMPLOYED_woe,EXT_SOURCE_2_woe,ORGANIZATION_TYPE_woe,OCCUPATION_TYPE_woe,DAYS_BIRTH_woe,AMT_CREDIT_woe
0,1,0.141677,1.105538,0.258284,0.482864,0.176570,0.222740,0.327196,0.182054
12,0,0.141677,1.105538,-0.174849,-0.510152,-0.005567,0.222740,0.149843,-0.217545
13,0,0.141677,-0.733450,-0.174849,0.012865,0.176570,0.222740,-0.115544,-0.236493
18,0,0.141677,-0.189907,-0.577772,-0.510152,-0.254422,0.222740,-0.115544,-0.217545
24,0,0.141677,-0.189907,0.258284,-0.510152,0.176570,0.222740,0.149843,0.182054
...,...,...,...,...,...,...,...,...,...
307495,0,-0.396824,-0.189907,-0.577772,0.482864,0.176570,-0.324802,-0.485361,-0.016153
307498,0,0.141677,-0.189907,-0.174849,-0.510152,0.176570,0.222740,-0.115544,-0.236493
307508,0,-0.396824,0.612624,-0.577772,0.012865,-0.672326,-0.324802,-0.115544,-0.016153
307509,1,0.141677,-0.733450,-0.577772,0.012865,0.176570,0.222740,0.149843,0.182054


In [30]:
#create the X, y parts of data for train and test
y_train = train_woe.loc[:,'STATUS']
X_train = train_woe.loc[:,train_woe.columns != 'STATUS']
y_test = test_woe.loc[:,'STATUS']
X_test = test_woe.loc[:,train_woe.columns != 'STATUS']

#create a logistic regression model object
lr = LogisticRegression()
lr.fit(X_train, y_train)
print("beta coefficients:")
print(lr.coef_)
print("alpha")
print(lr.intercept_)

beta coefficients:
[[0.55297102 0.874467   0.45848257 0.80057013 0.57790652 0.39097372
  0.2559806  0.63980556]]
alpha
[-2.52321225]


In [31]:
# generate a card from the model and bins
card = sc.scorecard(bins, lr, X_train.columns, points0 = 600, odds0 = 1/20, pdo = 20,
      basepoints_eq0 = True)

pprint.pprint(card)

{'AMT_CREDIT':       variable                   bin  points
43  AMT_CREDIT       [-inf,200000.0)    77.0
44  AMT_CREDIT   [200000.0,650000.0)    70.0
45  AMT_CREDIT   [650000.0,850000.0)    74.0
46  AMT_CREDIT  [850000.0,1350000.0)    78.0
47  AMT_CREDIT       [1350000.0,inf)    85.0,
 'DAYS_BIRTH':        variable                  bin  points
116  DAYS_BIRTH      [-inf,-19500.0)    77.0
117  DAYS_BIRTH  [-19500.0,-14000.0)    74.0
118  DAYS_BIRTH  [-14000.0,-11000.0)    72.0
119  DAYS_BIRTH       [-11000.0,inf)    71.0,
 'DAYS_EMPLOYED':          variable                bin  points
10  DAYS_EMPLOYED     [-inf,-4200.0)    81.0
11  DAYS_EMPLOYED  [-4200.0,-2000.0)    76.0
12  DAYS_EMPLOYED  [-2000.0,-1400.0)    73.0
13  DAYS_EMPLOYED      [-1400.0,inf)    70.0,
 'EXT_SOURCE_2':         variable                       bin  points
48  EXT_SOURCE_2                [-inf,0.2)    49.0
49  EXT_SOURCE_2                 [0.2,0.4)    62.0
50  EXT_SOURCE_2  [0.4,0.6000000000000001)    73.0
51  EXT_

In [ ]:
#Check the score card for occupation_type 
#See if drivers are discriminated against 